In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from scipy import interpolate
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
import pyart
import cartopy.crs as ccrs
from IPython.display import HTML
from io import StringIO
%matplotlib inline
# %matplotlib notebook
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
# Function definitions
def roundPartial(value, resolution, decimals=4):
    return np.around(np.round(value / resolution) * resolution, decimals=decimals)


def mtokm(val,pos):
    """Convert m to km for formatting axes tick labels"""
    val=val/1000.0
    return '%i' % val



In [ ]:
# Read in the gridded radar data
radar_name = 'KGWX'
radar_type= 'NEXRAD'
# For 04/29/16 case (IOP4B)
date = '0429'
radar_start_datetimestamp = '20160429212000'
radar_end_datetimestamp = '20160430000000'
height = 1000.

# Create datetime objects for start and end times
datetime_start = datetime.strptime(radar_start_datetimestamp, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(radar_end_datetimestamp, '%Y%m%d%H%M%S')

radar_basedir = \
    '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/2016/NEXRAD/IOP_4B/GWX/'
# FIXME: need to standardize naming of radar data directories
# radar_basedir = os.path.join(radar_basedir, '{}/{}'.format(date, radar_name[1:]))
gridded_radar_dir = os.path.join(radar_basedir, 'gridded')

radar_start_timestamp = datetime_start.strftime('%Y%m%d%H%M')
radar_end_timestamp = datetime_end.strftime('%Y%m%d%H%M')
gridded_radar_filename = '{}_{}_{}_z{:d}_gridded_interp_retr.nc'.format(radar_name, radar_start_timestamp,
                                                                        radar_end_timestamp, int(height))
gridded_radar_filepath = os.path.join(gridded_radar_dir, gridded_radar_filename)

gridded_radar_ds = xr.open_dataset(gridded_radar_filepath)

In [ ]:
plot_dir = os.path.join(gridded_radar_dir, 'plots')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

In [ ]:
# Read in PIPS data
deployment = 'IOP4B_D1_2016'
PIPS_list = ['PIPS1B', 'PIPS2A', 'PIPS2B']
PIPS_data_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15'

PIPS_ds_list = []
PIPS_locs = []

for PIPS in PIPS_list:
    PIPS_filename = 'parsivel_combined_{}_{}_60s.nc'.format(deployment, PIPS)
    PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
    PIPS_ds = xr.load_dataset(PIPS_filepath)
    PIPS_ds_list.append(PIPS_ds)
    PIPS_loc = eval(PIPS_ds.location)
    PIPS_locs.append(PIPS_loc)

In [ ]:
for PIPS_ds in PIPS_ds_list:
    print(PIPS_ds["KGWX_beam_height_at_PIPS"].mean())

In [ ]:
# Find PIPS x, y location by interpolating to its lat/lon point
gridded_radar_latlon_ds = gridded_radar_ds.swap_dims({'x': 'lon', 'y': 'lat'})
print(gridded_radar_latlon_ds)

radar_at_PIPS_list = []
PIPS_xy_list = []

for PIPS_loc in PIPS_locs:
    PIPS_lat = PIPS_loc[0]
    PIPS_lon = PIPS_loc[1]
    radar_at_PIPS_da = gridded_radar_latlon_ds.interp(lat=PIPS_lat, lon=PIPS_lon)
    print(PIPS_lat, PIPS_lon)
    PIPS_x = radar_at_PIPS_da['x'].values.item()
    PIPS_y = radar_at_PIPS_da['y'].values.item()
    PIPS_xy = (PIPS_x, PIPS_y)
    PIPS_xy_list.append(PIPS_xy)
    print(PIPS_x, PIPS_y)
    radar_at_PIPS_list.append(radar_at_PIPS_da)

In [ ]:
# Choose a subset of times to keep animation size down
anim_start = '2016-04-29T21:20'
anim_end = '2016-04-29T22:30'

In [ ]:
# Plot reflectivity for time-interpolated grid
var_da = gridded_radar_ds['reflectivity_masked'].sel(time=slice(anim_start, anim_end))
xplt = var_da.coords['x']
yplt = var_da.coords['y']

clevels =np.arange(0., 61., 1.)
norm = cm.colors.Normalize(vmin=0., vmax=60.)

import matplotlib.animation as animation
fig, ax = plt.subplots(figsize=(8, 8))

ims = []
for i, var in enumerate(var_da):
    time = var.coords['time_seconds'].values.item()
    ci = ax.contourf(xplt, yplt, var.squeeze(), levels=clevels, 
                     cmap='pyart_HomeyerRainbow', norm=norm)
    # Plot PIPS location
    for PIPS, PIPS_xy in zip(PIPS_list, PIPS_xy_list):
        PIPS_x = PIPS_xy[0]
        PIPS_y = PIPS_xy[1]
        ax.plot([PIPS_x], [PIPS_y], 'k*')
        #ax.text([PIPS_x], [PIPS_y], str(PIPS_list['val'])
    if i == 0.:
        fig.colorbar(ci, ax=ax)
        ax.set_xlim(25000., 100000.)
        ax.set_ylim(-50000., 25000.)
        ax.set_aspect('equal')
    ims.append(ci.collections)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)

plt.close()
HTML(ani.to_jshtml())
# ani.save('test.mp4')

In [ ]:
# Save above animation to disk
ani_filename = 'interp_masked_ref_{}_{}_z{:d}.mp4'.format(anim_start, anim_end, int(height))
ani_filepath = os.path.join(plot_dir, ani_filename)

ani.save(ani_filepath)

In [ ]:
var_da = gridded_radar_ds['D0'].sel(time=slice(anim_start, anim_end))

clevels =np.arange(0., 6., 0.1)
norm = cm.colors.Normalize(vmin=0., vmax=6.)

import matplotlib.animation as animation
fig, ax = plt.subplots(figsize=(8, 8))

ims = []
for i, var in enumerate(var_da):
    time = var.coords['time_seconds'].values.item()
    ci = ax.contourf(xplt, yplt, var.squeeze(), 
                     levels=clevels, 
                     cmap='pyart_HomeyerRainbow', norm=norm)
    # Plot PIPS location
    for PIPS, PIPS_xy in zip(PIPS_list, PIPS_xy_list):
        PIPS_x = PIPS_xy[0]
        PIPS_y = PIPS_xy[1]
        ax.plot([PIPS_x], [PIPS_y], 'k*')
    if i == 0.:
        fig.colorbar(ci, ax=ax)
        ax.set_xlim(50000., 120000.)
        ax.set_ylim(50000., 120000.)
        ax.set_aspect('equal')
    ims.append(ci.collections)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)

plt.close()
HTML(ani.to_jshtml())
# plt.show()

In [ ]:
# Save above animation to disk
ani_filename = 'D0_{}_{}_z{:d}.mp4'.format(anim_start, anim_end, int(height))
ani_filepath = os.path.join(plot_dir, ani_filename)

ani.save(ani_filepath)

In [ ]:
# Plot differential reflectivity for time-interpolated grid
var_da = gridded_radar_ds['differential_reflectivity_masked'].sel(time=slice(anim_start, anim_end))
xplt = var_da.coords['x']
yplt = var_da.coords['y']

clevels =np.arange(0., 6., 0.1)
norm = cm.colors.Normalize(vmin=0., vmax=6.)

import matplotlib.animation as animation
fig, ax = plt.subplots(figsize=(8, 8))

ims = []
for i, var in enumerate(var_da):
    time = var.coords['time_seconds'].values.item()
    ci = ax.contourf(xplt, yplt, var.squeeze(), levels=clevels, 
                     cmap='pyart_HomeyerRainbow', norm=norm)
    # Plot PIPS location
    for PIPS, PIPS_xy in zip(PIPS_list, PIPS_xy_list):
        PIPS_x = PIPS_xy[0]
        PIPS_y = PIPS_xy[1]
        ax.plot([PIPS_x], [PIPS_y], 'k*')
        #ax.text([PIPS_x], [PIPS_y], str(PIPS_list['val'])
    if i == 0.:
        fig.colorbar(ci, ax=ax)
        ax.set_xlim(25000., 100000.)
        ax.set_ylim(-50000., 25000.)
        ax.set_aspect('equal')
    ims.append(ci.collections)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)

plt.close()
HTML(ani.to_jshtml())

In [ ]:
# Save above animation to disk
ani_filename = '1km_AGL_ZDR_{}_{}_z{:d}.mp4'.format(anim_start, anim_end, int(height))
ani_filepath = os.path.join(plot_dir, ani_filename)

ani.save(ani_filepath)

In [ ]:
tmatrix_path = '/Users/dawson29/Projects/pyPIPS/tmatrix/S-Band/SCTT_RAIN_fw100.dat'
PIPS_ds_new_list = []
dualpol_dict_PIPS_list = []
for PIPS, PIPS_ds, radar_at_PIPS_da in zip(PIPS_list, PIPS_ds_list, radar_at_PIPS_list):
    dD = PIPS_ds['max_diameter'] - PIPS_ds['min_diameter']
    dualpol_dict_PIPS = dualpol.calpolrain_bulk_xr(10.7, tmatrix_path, PIPS_ds['ND_qc'], dD, 
                                                   diameter_bin_name='diameter_bin')
    dualpol_dict_PIPS_list.append(dualpol_dict_PIPS)
    gridded_varnames = list(radar_at_PIPS_da.keys())
    PIPS_ds_new = PIPS_ds.copy()
    PIPS_ds_new = PIPS_ds_new.drop_dims('gridded_fields_KGWX')
    PIPS_ds_new.coords['gridded_fields_KGWX'] = gridded_varnames
    new_radar_at_PIPS_da = radar_at_PIPS_da.to_array(dim='gridded_fields_{}'.format(radar_name), 
                                                     name='gridded_{}_at_PIPS'.format(radar_name))
    # Interpolate to PIPS times
    new_radar_at_PIPS_da = new_radar_at_PIPS_da.interp_like(PIPS_ds_new)
    PIPS_ds_new['gridded_{}_at_PIPS'.format(radar_name)] = new_radar_at_PIPS_da
    
    PIPS_ds_new_list.append(PIPS_ds_new)
    PIPS_filename = 'parsivel_combined_{}_{}_60s.nc'.format(deployment, PIPS)
    PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
    PIPS_ds_new.to_netcdf(PIPS_filepath)

In [ ]:
# Made this a bit easier to deal with and less error-prone. 
# Now elect by PIPS name instead of trying to remember the index
PIPS_to_plot = 'PIPS1B' # Pick the particular PIPS you prefer to plot, and pick a peck of pickled peppers while
# you're at it ;)
PIPS_index = PIPS_list.index(PIPS_to_plot)
print("Index of {} in the list is {:d}".format(PIPS_to_plot, PIPS_index))

retrieval_tag = 'Z01'

PIPS_xy = PIPS_xy_list[PIPS_index]
PIPS_x = PIPS_xy[0]
PIPS_y = PIPS_xy[1]

PIPS_ds_new = PIPS_ds_new_list[PIPS_index]
# radar_at_PIPS_da contains the gridded radar at the top of the sorting layer interpolated to the PIPS 
# horizontal location
radar_at_PIPS_da = radar_at_PIPS_list[PIPS_index]

dD = PIPS_ds['max_diameter'] - PIPS_ds['min_diameter']
dualpol_dict_PIPS = dualpol_dict_PIPS_list[PIPS_index]


In [ ]:
plot_start = '2016-04-29T21:20'
plot_end = '2016-04-30T00:00'

In [ ]:
ZH_PIPS = dualpol_dict_PIPS['REF']
ZH_PIPS.plot(xlim=(plot_start, plot_end), color='k')
gridded_ZH = PIPS_ds_new['gridded_KGWX_at_PIPS'].sel(gridded_fields_KGWX='reflectivity_masked')
gridded_ZH.plot(xlim=(plot_start, plot_end), color='g')
PIPS_ds['KGWX_at_PIPS'].sel(fields_KGWX='REF').plot(xlim=(plot_start, plot_end), color='purple')

figname_2 = 'KGWX_REF_{}_{}.png'.format(date, PIPS_to_plot)
figpath = os.path.join(plot_dir, figname_2)
plt.savefig(figpath, dpi=300, bbox_inches='tight')

In [ ]:
Dm_PIPS = PIPS_ds_new['Dm43_qc'] * 1000.
Dm_PIPS.plot(xlim=(plot_start, plot_end), color='k')
gridded_Dm = PIPS_ds_new['gridded_KGWX_at_PIPS'].sel(gridded_fields_KGWX='Dm_{}'.format(retrieval_tag))
gridded_Dm.plot(xlim=(plot_start, plot_end), color='g')
PIPS_ds_new['KGWX_at_PIPS'].sel(fields_KGWX='Dm_{}'.format(retrieval_tag)).plot(xlim=(plot_start, plot_end), 
                                                                                color='purple')

figname_2 = 'KGWX_Dm_{}_{}.png'.format(date, PIPS_to_plot)
figpath = os.path.join(plot_dir, figname_2)
plt.savefig(figpath, dpi=300, bbox_inches='tight')

In [ ]:
ZDR_PIPS = dualpol_dict_PIPS['ZDR']
ZDR_PIPS.plot(xlim=(plot_start, plot_end), color='k')
gridded_ZDR = PIPS_ds_new['gridded_KGWX_at_PIPS'].sel(gridded_fields_KGWX='differential_reflectivity_masked')
gridded_ZDR.plot(xlim=(plot_start, plot_end), color='g')
PIPS_ds_new['KGWX_at_PIPS'].sel(fields_KGWX='ZDR').plot(xlim=(plot_start, plot_end),
                                                        color='purple')

figname_3 = 'KGWX_ZDR_{}_{}.png'.format(date, PIPS_to_plot)
figpath = os.path.join(plot_dir, figname_3)
plt.savefig(figpath, dpi=300, bbox_inches='tight')